In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import holidays
from sklearn.preprocessing import LabelEncoder
import  numpy as np
import sqlite3
import os


In [ ]:
path=pd.read_csv(r"/database/data\retail_data\products.csv")
df=pd.DataFrame(path)


In [ ]:
# Connect to SQLite database
db_path = r"C:\Users\kingd\Ennovar\database\database.db"
conn = sqlite3.connect(db_path)

# Read data from database tables
x = pd.read_sql_query("SELECT * FROM transactions", conn)
y = pd.read_sql_query("SELECT * FROM products", conn)

transactions = pd.DataFrame(x)
products = pd.DataFrame(y)

conn.close()

In [ ]:
transactions.info()

In [ ]:
products.info()

In [ ]:
merged = pd.merge(df1, df2, on="Product ID", how="inner")
merged=merged[merged['Currency']=='USD']
merged

### Explore dataset

In [ ]:
df=merged[['Date','Sub Category','Transaction Type','Quantity','Invoice Total']].copy()
df=df.rename(columns={'Sub Category':'Category'})
df=df.rename(columns={'Invoice Total':'Total sales (Dollar)'})
df=df[df['Transaction Type']=='Sale']
df['Date']=pd.to_datetime(df['Date'])
df['Date']=df['Date'].dt.date

In [ ]:
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date'].dt.year.nunique()

In [ ]:
daily_df = df.groupby(['Date','Category'])['Quantity'].sum().reset_index()
daily_df['Date']=pd.to_datetime(daily_df['Date'])
daily_df

In [ ]:
daily_df['Category'].nunique()

### Sales in 2023

In [ ]:
df_2023=daily_df[daily_df['Date'].dt.year==2023]
#top 5 sales item
top_5=df_2023.groupby('Category')['Quantity'].sum().nlargest(5).reset_index()
top_5

### The sales pattern of most five sale Items in 2023

In [ ]:
my_holidays = holidays.US()
for item in top_5['Category'].unique():

    item_df = df_2023[df_2023['Category'] == item].copy()

    plt.figure(figsize=(15, 6))
    plt.plot(item_df['Date'], item_df['Quantity'])

    item_df['Holiday_Name'] = item_df['Date'].apply(lambda d: my_holidays.get(d, None))
    holiday_df = item_df[item_df['Holiday_Name'].notna()]

    if not holiday_df.empty:
        for _, row in holiday_df.iterrows():
            plt.scatter(row['Date'], row['Quantity'], color='red', s=90, label=row['Holiday_Name'])
            plt.text(row['Date'], row['Quantity'], row['Holiday_Name'], fontsize=8, color='red', ha='left', va='bottom')

    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

    plt.title(f"{item} — 2023")
    plt.xlabel("Date")
    plt.ylabel("Quantity")
    plt.xticks(rotation=45, ha='right')

    if not holiday_df.empty:
        handles, labels = plt.gca().get_legend_handles_labels()
        unique = dict(zip(labels, handles))
        plt.legend(unique.values(), unique.keys())   # remove duplicates

    plt.tight_layout()
    plt.show()


### Pants_and_Jeans

### 1.The pattern sale of Pants_and_Jeans top peak month

In [ ]:
my_holidays = holidays.US()
# Filter Pants & Jeans
Pants_and_Jeans_df = df_2023[df_2023['Category'] == 'Pants and Jeans'].copy()

# Add holiday names
Pants_and_Jeans_df['Holiday_Name'] = Pants_and_Jeans_df['Date'].apply(
    lambda d: my_holidays.get(d, None)
)

# Extract only rows where there is a holiday
holidays = Pants_and_Jeans_df.dropna(subset=['Holiday_Name'])

# Plot
plt.figure(figsize=(20, 6))
plt.plot(
    Pants_and_Jeans_df['Date'],
    Pants_and_Jeans_df['Quantity'],
    linewidth=2
)

# Add holiday markers
plt.scatter(
    holidays['Date'],
    holidays['Quantity'],
    s=120,
    marker='o',
    color='red'
)

# Add labels for each holiday marker
for _, row in holidays.iterrows():
    plt.annotate(
        row['Holiday_Name'],
        (row['Date'], row['Quantity']),
        xytext=(0, 12),
        textcoords='offset points',
        ha='center',
        fontsize=9,
        color='red'
    )

# Labels and formatting
plt.xlabel('Date')
plt.ylabel('Quantity')
plt.title('Pants and Jeans — 2023')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


In [ ]:
jan = Pants_and_Jeans_df[Pants_and_Jeans_df['Date'].dt.month == 1]
sep = Pants_and_Jeans_df[Pants_and_Jeans_df['Date'].dt.month == 9]
dec = Pants_and_Jeans_df[Pants_and_Jeans_df['Date'].dt.month == 12]

fig, axes = plt.subplots(3, 1, figsize=(16, 12), sharey=False)

datasets = [(jan, "January"), (sep, "September"), (dec, "December")]

for ax, (df, month_name) in zip(axes, datasets):
    ax.plot(df['Date'], df['Quantity'], marker='o')
    df['Holiday_Name'] = df['Date'].apply(lambda d: my_holidays.get(d, None))
    holiday_df = df[df['Holiday_Name'].notna()]   # Keep only holidays

    if not holiday_df.empty:
        ax.scatter(
            holiday_df['Date'],
            holiday_df['Quantity'],
            color='red',
            s=80,
            label='Holiday'
        )

        for _, row in holiday_df.iterrows():
            ax.text(
                row['Date'],
                row['Quantity'],
                f"{row['Holiday_Name']}",
                fontsize=8,
                color='red',
                ha='left',
                va='bottom'
            )
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))

    ax.set_title(f"Pants and Jeans — {month_name} 2023")
    ax.set_xlabel("Date")
    ax.set_ylabel("Quantity")
    ax.tick_params(axis='x', rotation=45)

    if not holiday_df.empty:
        ax.legend()

plt.tight_layout()
plt.show()

### Check the trend at Weekly

In [ ]:
x = df_2023.copy()
x['Date'] = pd.to_datetime(x['Date'])
x = x.set_index('Date')

weekly_df = (
    x[x['Category']=='Pants and Jeans']
    .resample('w')
    .sum()
).reset_index()

plt.figure(figsize=(12,5))
plt.plot(weekly_df['Date'], weekly_df['Quantity'], marker='o')

plt.xlabel('Date')
plt.ylabel('Quantity')
plt.title('Pants and Jeans in 2023 (weekly_df)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
x = df.copy()
x['Date'] = pd.to_datetime(x['Date'])
x = x.set_index('Date')

weekly_df = (
    x[x['Category']=='Pants and Jeans']
    .resample('Q')
    .sum()
).reset_index()

plt.figure(figsize=(12,5))
plt.plot(weekly_df['Date'], weekly_df['Quantity'], marker='o')

plt.xlabel('Date')
plt.ylabel('Quantity')
plt.title('Pants and Jeans in 2023-2025 (weekly_df)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Check the trend at Quarterly

In [ ]:
df_2023

In [ ]:
x = df_2023.copy()
x['Date'] = pd.to_datetime(x['Date'])
x = x.set_index('Date')

quarterly_df = (
    x[x['Category']=='Pants and Jeans']
    .resample('Q')
    .sum()
).reset_index()

plt.figure(figsize=(12,5))
plt.plot(quarterly_df['Date'], quarterly_df['Quantity'], marker='o')

plt.xlabel('Date')
plt.ylabel('Quantity')
plt.title('Pants and Jeans in 2023 (Quarterly)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
y = df.copy()
y['Date'] = pd.to_datetime(y['Date'])
y = y.set_index('Date')

quarterly_df = (
    y[y['Category']=='Pants and Jeans']
    .resample('Q')
    .sum()
).reset_index()

plt.figure(figsize=(12,5))
plt.plot(quarterly_df['Date'], quarterly_df['Quantity'], marker='o')

plt.xlabel('Date')
plt.ylabel('Quantity')
plt.title('Pants and Jeans in 2023 - 2025 (Quarterly)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
merged

### Train and test data

In [ ]:
df=merged[['Date','Category','Size','Color_x','Discount','Quantity','Sub Category']].copy()

In [ ]:
df

In [ ]:
#Handle null and duplicate
null_count=df.isnull().sum()
df=df.dropna(subset=['Size', 'Color_x'])
null_count

In [ ]:
# df['Date']=pd.to_datetime(df['Date'])
# df['Date']=df['Date'].dt.date
# df['Date']=pd.to_datetime(df['Date'])
# allowed_sizes = ['M', 'L', 'S', 'XL', 'XXL']
# df = df[df['Size'].isin(allowed_sizes)]
# df =  df.groupby(['Date','Category','Size','Color_x'])[['Discount','Quantity']].sum() .reset_index()
# df = df.rename(columns={'Quantity': 'Total sales'})

In [ ]:
#feature engineer
import holidays
df['Date'] = pd.to_datetime(df['Date'])
df['day_of_year'] = df['Date'].dt.dayofyear
df['month'] = df['Date'].dt.month
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['day_of_week'] = df['Date'].dt.dayofweek
df['week_of_year'] = df['Date'].dt.isocalendar().week.astype(int)
df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)
df['is_month_end'] = df['Date'].dt.is_month_end.astype(int)
df['is_month_start'] = df['Date'].dt.is_month_start.astype(int)

for col in ['Category','Size','Color_x','Sub Category']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Sort by date
df = df.sort_values('Date')

# Lag features
df['lag_1'] = df['Quantity'].shift(1) #sales of one day ago
df['lag_7'] = df['Quantity'].shift(7) #sales of seven days ago
df['lag_30'] = df['Quantity'].shift(30) #sales of one month ago
df['rolling_7'] = df['Quantity'].rolling(7).mean() #The avg sales of seven sales
df['rolling_30'] = df['Quantity'].rolling(30).mean()  #The avg sales of 30 sales
df=df.dropna()

# Holiday
country_holidays = holidays.US()
df['is_holiday'] = df['Date'].isin(country_holidays).astype(int)
df['is_holiday_tomorrow']=df['Date'].shift(-1).isin(country_holidays).astype(int)
df['is_holiday_yesterday']=df['Date'].shift(1).isin(country_holidays).astype(int)
#big sale events
df['zscore'] = (df['Quantity'] - df['Quantity'].mean()) / df['Quantity'].std()
df['big_sale_event'] = (df['zscore'] > 2.5).astype(int)
df = df.drop(columns=['month', 'zscore'])

In [ ]:
df.info()

In [ ]:
df

In [ ]:
# Train/Test split
train = df[df['Date'] < '2025-1-01']
test = df[df['Date'] >= '2025-1-01']

X_train = train.drop(['Date','Quantity'], axis=1)
y_train = train['Quantity']

X_test = test.drop(['Date','Quantity'], axis=1)
y_test = test['Quantity']

### Model

### 1. XGBoost

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error
model = XGBRegressor(
    n_estimators=10000,        # number of trees
    learning_rate=0.05,     # shrinkage rate
    max_depth=6,            # tree depth
    min_child_weight=1,     # minimum data in child node
    subsample=0.8,          # row sampling
    colsample_bytree=0.8,   # feature sampling
    gamma=0,                # min loss reduction (split)
    reg_alpha=0.0,          # L1 regularization
    reg_lambda=1.0,         # L2 regularization
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
pred_value = model.predict(X_test)

mae = mean_absolute_error(y_test, pred_value)
mape = mean_absolute_percentage_error(y_test, pred_value) * 100
print("Test MAE:", mae)
print("Test MAPE:", mape)

### 2. Catboost

In [ ]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(
    iterations=10000,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3.0,
    loss_function='RMSE',
    random_state=42,
    bootstrap_type='Bayesian',   # keep Bayesian
    bagging_temperature=1.0,     # controls Bayesian sampling
    thread_count=-1,
    verbose=False
)
model.fit(X_train, y_train)
pred_value = model.predict(X_test)
mae = mean_absolute_error(y_test, pred_value)
mape = mean_absolute_percentage_error(y_test, pred_value) * 100
print("Test MAE:", mae)
print("Test MAPE:", mape)

### Improve the accuracy: 
1. Feature Engineering: create features that reveal trends, seasonality, cycles, events, and context: Done
2. Better Train–Test Splitting (Time-Aware): NEVER use shuffle=True in time-series.
3. Handle Outliers in Sales Data
4. Target Transformation
5. Reduce Noise in Features
6. Train Separate Models per Product / Category
7. Cross-validation for time-series: Use TimeSeriesSplit instead of simple train/test
8. Hyperparameter:
    - max_depth: More depth → more complex trees → better learning (but risk overfitting)
    - min_child_weight:Lower values → more sensitive splits → captures rare patterns.
    - learning_rate + n_estimators: Lower learning rate → more accurate but slower
    - subsample & colsample_bytree: These reduce overfitting and increase generalization.
    - gamma: helps prune unnecessary splits.
    - reg_alpha & reg_lambda: Regularization keeps trees from overfitting

In [ ]:
# Improve the accuracy: 
1. 